In [1]:
import pandas as pd
import time
from river import cluster
from river import metrics
from river.stream import iter_pandas

In [2]:
data = pd.read_csv("../datasets/agr_a_20k.csv")
features = data.columns[:-2]
stream = iter_pandas(X=data[features], y=data['class'])


clustream = cluster.CluStream(time_window=10,
                              max_micro_clusters=500,
                              n_macro_clusters=10,
                              time_gap=500,
                              seed=0,
                              halflife=0.4)

start_1 = time.time()
for sample_cnt, (x, y_true) in enumerate(stream):
    clustream.learn_one(x)
end_1 = time.time()
print(end_1 - start_1)
print(clustream._memory_usage)

51.34182095527649
2.15 MB


In [3]:
clustream.centers

{0: defaultdict(functools.partial(<bound method Random.gauss of <random.Random object at 0x133c26210>>, 0, 1),
             {'hvalue': 0.9417154046806644,
              'elevel': -1.3965781047011498,
              'salary': -0.6797144480784211,
              'zipcode': 0.3705035674606598,
              'hyears': -1.016348894188071,
              'car': -0.07212002278507135,
              'commission': 0.17919648727485687,
              'age': -0.8310992152709882}),
 1: defaultdict(functools.partial(<bound method Random.gauss of <random.Random object at 0x133c26210>>, 0, 1),
             {'hvalue': -1.3090373644593587,
              'elevel': 0.1938877412491041,
              'salary': 0.9932497035351971,
              'zipcode': -0.6469816305475048,
              'hyears': -0.33366798379566953,
              'car': 1.6456717605826776,
              'commission': -0.5588897658482705,
              'age': -0.514156663052727}),
 2: defaultdict(functools.partial(<bound method Random.gauss 

In [4]:
clustream.predict_one(x)

7